In [1]:
#!/usr/bin/env python
# coding: utf-8
import sys
import scanpy as sc
import anndata
import pandas as pd
import numpy as np
import os
import gc
import decoupler as dc
import regex as re
import math
os.chdir("/Users/ricardoramirez/Dropbox/PhD/Research/MOFAcell")

In [2]:
sc_dat = sc.read_h5ad(filename = "/Volumes/RicoData2/MI_project/MI_revisions/HCA_submission/snrnaseq/hca/hca_rnasamples.h5ad")

In [3]:
sc_dat = sc_dat[~sc_dat.obs["cell_state"].isna().values]

In [36]:
# Next filter out cells that are part of analyzed cells
cts = ["CM","Fib","Endo","Myeloid"]
degs = {}

In [37]:
for ct in cts:
    print(ct)
    ct_dat = sc_dat[sc_dat.obs["cell_type"].values == ct].copy()
    sc.pp.normalize_total(ct_dat, target_sum=1e4)
    sc.pp.log1p(ct_dat)
    ct_dat.raw = ct_dat
    sc.tl.rank_genes_groups(ct_dat, groupby="cell_state",use_raw=True)
    mrkrs = sc.get.rank_genes_groups_df(ct_dat, group = None)
    mrkrs["ct"] = ct
    degs[ct] = mrkrs

CM
Fib
Endo
Myeloid


In [41]:
deg_df = pd.concat(degs)
deg_df.to_csv("./data_MI/cell_states_mrkrs_ttest.csv")

In [40]:
deg_df

group       names     scores  logfoldchanges  \
CM      0              healthy_CM        FHL2  83.354034        1.028467   
        1              healthy_CM  AC022034.3  80.016144        1.476064   
        2              healthy_CM   LINC02208  76.791519        1.430567   
        3              healthy_CM      TNNI3K  74.681068        0.909300   
        4              healthy_CM       FGF12  69.702843        0.933749   
...                           ...         ...        ...             ...   
Myeloid 145625  CCL18_Macrophages       LYVE1 -25.029985       -1.590276   
        145626  CCL18_Macrophages       SCN9A -28.019337       -2.220674   
        145627  CCL18_Macrophages       TGFBI -29.482315       -1.867936   
        145628  CCL18_Macrophages      P2RY14 -32.249977       -2.834095   
        145629  CCL18_Macrophages       F13A1 -36.369732       -2.336515   

                        pvals      pvals_adj       ct  
CM      0        0.000000e+00   0.000000e+00       CM  
        1        0.000000e+00   0.000000e+00       CM  
        2        0.000000e+00   0.000000e+00       CM  
        3        0.000000e+00   0.000000e+00       CM  
        4        0.000000e+00   0.000000e+00       CM  
...                       ...            ...      ...  
Myeloid 145625  2.409392e-126  3.693471e-123  Myeloid  
        145626  1.052262e-158  2.189156e-155  Myeloid  
        145627  4.304065e-170  1.044668e-166  Myeloid  
        145628  2.750455e-210  1.144425e-206  Myeloid  
        145629  3.357069e-235  4.888900e-231  Myeloid  

[553394 rows x 7 columns]